# Segmenting and Clustering Postal Codes in Downtown Toronto

**This is Part 3 of the Segmenting and Clustering Neighborhoods in Toronto Assignment**

This notebook focuses on clustering the venues located on Downtown Toronto using KMeans.


Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [1]:
# Uncomment if not yet installed
# !pip install geopy
# !pip install folium

# Import necessary libraries and data

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Import the data that we saved in Part 2.

In [3]:
dwnToronto_data = pd.read_csv('neighborhoods_part_2.csv')
# We don't need the first column (titled Unnamed: 0) so we'll just drop it. 
# The Unnamed: 0 column is simply the index of the dataframe prior to saving it
dwnToronto_data.drop('Unnamed: 0', axis =1, inplace = True)

# Display the first 5 rows 
dwnToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Exploring Downtown Toronto

As mentioned on the outset, the focus of this notebook is to cluster the neighborhoods in Downtown Toronto. So, let's filter the rows that have Downtown Toronto as Borough and name it dwnToronto

In [4]:
dwnToronto = dwnToronto_data[dwnToronto_data.Borough == 'Downtown Toronto']

# reset the index to 0, 1, ...
dwnToronto.reset_index(inplace = True)
# drop the first column
dwnToronto.drop('index', axis =1, inplace = True)

dwnToronto

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


Get the total number of neighborhood in Downtown Toronto. Postal Codes that have multiple neighborhood are separated by a comma (,) so to count the total number of neighborhood we first need to split the neighborhood and then total it. 

In [5]:
def neighborhood_count(neighborhood_list): 
  return len(neighborhood_list.split(','))

total_neighborhood = dwnToronto['Neighborhood'].apply(neighborhood_count).sum()

print('There are {} neighborhoods and {} unique postal codes in Downtown Toronto'.format(total_neighborhood, 
                                                                                         dwnToronto['PostalCode'].nunique()))

There are 36 neighborhoods and 17 unique postal codes in Downtown Toronto


<a id='item1'></a>


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [6]:
address = 'Downtown Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


#### Create a map of Downtown Toronto with neighborhoods superimposed on top.


In [7]:
# create map of New York using latitude and longitude values
map_dwn_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dwnToronto_data['Latitude'], dwnToronto_data['Longitude'], dwnToronto_data['Borough'], dwnToronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dwn_toronto)  
    
map_dwn_toronto

#### Define Foursquare Credentials and Version


In [8]:
CLIENT_ID = 'TOFMY3V5HDI32TJKGHT0I5ZJNCRXRQY0YFTCGZAOY0DWPM1O' # your Foursquare ID
CLIENT_SECRET = 'FHTRZAAAQTFT4FFSE2UOO1YQCGT5HUTHSQTHDXIXXE3QHT4B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TOFMY3V5HDI32TJKGHT0I5ZJNCRXRQY0YFTCGZAOY0DWPM1O
CLIENT_SECRET:FHTRZAAAQTFT4FFSE2UOO1YQCGT5HUTHSQTHDXIXXE3QHT4B


#### Let's explore the first Postal Code in our dataframe.


Get the first postal code in dwnToronto


In [9]:
dwnToronto.loc[0, 'PostalCode']

'M5A'

In [10]:
neighborhood_latitude = dwnToronto.loc[0, 'Latitude'] # postal code latitude value
neighborhood_longitude = dwnToronto.loc[0, 'Longitude'] # postal code longitude value

neighborhood_name = dwnToronto.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_postalcode = dwnToronto.loc[0, 'PostalCode'] # neighborhood name
print('Latitude and longitude values of {} which contains the neighborhoods {} are {}, {}.'.format(neighborhood_postalcode,
                                                                                                   neighborhood_name, 
                                                                                                   neighborhood_latitude, 
                                                                                                   neighborhood_longitude))

Latitude and longitude values of M5A which contains the neighborhoods Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in M5A Postal Code within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [11]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TOFMY3V5HDI32TJKGHT0I5ZJNCRXRQY0YFTCGZAOY0DWPM1O&client_secret=FHTRZAAAQTFT4FFSE2UOO1YQCGT5HUTHSQTHDXIXXE3QHT4B&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Send the GET request and examine the resutls


In [12]:
results = requests.get(url).json()
# results

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-14-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


And how many venues were returned by Foursquare?


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in Downtown Toronto


#### Let's create a function to repeat the same process to all the postal codes in Downtown Toronto


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _dwnToronto_venues_.


In [17]:
# type your answer here
dwnToronto_venues = getNearbyVenues(names=dwnToronto['PostalCode'],
                                   latitudes=dwnToronto['Latitude'],
                                   longitudes=dwnToronto['Longitude']
                                  )

M5A
M5B
M5C
M5E
M5G
M6G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M4W
M4X
M5X
M4Y


#### Let's check the size of the resulting dataframe


In [18]:
print(dwnToronto_venues.shape)
dwnToronto_venues.head()

(1110, 7)


,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each postal code


In [19]:
dwnToronto_venues.groupby('Postal Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4W,4,4,4,4,4,4
M4X,44,44,44,44,44,44
M4Y,76,76,76,76,76,76
M5A,44,44,44,44,44,44
M5B,100,100,100,100,100,100
M5C,89,89,89,89,89,89
M5E,57,57,57,57,57,57
M5G,67,67,67,67,67,67
M5H,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues


In [20]:
print('There are {} uniques categories.'.format(len(dwnToronto_venues['Venue Category'].unique())))

There are 202 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [21]:
# one hot encoding
dwnToronto_onehot = pd.get_dummies(dwnToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postal Code column back to dataframe
dwnToronto_onehot['Postal Code'] = dwnToronto_venues['Postal Code'] 

# move Postal Code column to the first column
fixed_columns = [dwnToronto_onehot.columns[-1]] + list(dwnToronto_onehot.columns[:-1])
dwnToronto_onehot = dwnToronto_onehot[fixed_columns]

dwnToronto_onehot.head()

,Postal Code,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.


In [22]:
dwnToronto_onehot.shape

(1110, 203)

#### Next, let's group rows by postal code and by taking the mean of the frequency of occurrence of each category


In [23]:
dwnToronto_grouped = dwnToronto_onehot.groupby('Postal Code').mean().reset_index()
dwnToronto_grouped

,Postal Code,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4W,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

#### Let's confirm the new size


In [24]:
dwnToronto_grouped.shape

(17, 203)

#### Let's print each neighborhood along with the top 5 most common venues


In [25]:
num_top_venues = 5

for hood in dwnToronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = dwnToronto_grouped[dwnToronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
        venue  freq
0        Park  0.50
1  Playground  0.25
2       Trail  0.25
3     Airport  0.00
4      Museum  0.00


----M4X----
                venue  freq
0         Coffee Shop  0.07
1         Pizza Place  0.07
2  Italian Restaurant  0.05
3                Park  0.05
4                 Pub  0.05


----M4Y----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.07
2  Japanese Restaurant  0.07
3           Restaurant  0.04
4              Gay Bar  0.04


----M5A----
            venue  freq
0     Coffee Shop  0.18
1            Park  0.07
2             Pub  0.07
3          Bakery  0.07
4  Breakfast Spot  0.05


----M5B----
                 venue  freq
0          Coffee Shop  0.10
1       Clothing Store  0.09
2  Japanese Restaurant  0.03
3                Hotel  0.03
4       Cosmetics Shop  0.03


----M5C----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2      Restaurant  0.04
3  Clothing Store  0.03
4    Cocktail Bar 

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['Postal Code'] = dwnToronto_grouped['Postal Code']

for ind in np.arange(dwnToronto_grouped.shape[0]):
    postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dwnToronto_grouped.iloc[ind, :], num_top_venues)

postalcode_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4X,Coffee Shop,Pizza Place,Café,Italian Restaurant,Bakery,Park,Restaurant,Pub,General Entertainment,Beer Store
2,M4Y,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Mediterranean Restaurant,Hotel,Fast Food Restaurant
3,M5A,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Restaurant
4,M5B,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Hotel,Café,Japanese Restaurant,Diner,Electronics Store,Pizza Place


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the postal codes into 5 clusters.


In [28]:
# set number of clusters
kclusters = 5

dwnToronto_grouped_clustering = dwnToronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dwnToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [29]:
# add clustering labels
postalcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
postalcode_venues_sorted

dwnToronto_merged = dwnToronto

# merge dwnToronto_grouped with dwnToronto_data to add latitude/longitude for each postal codes
dwnToronto_merged = dwnToronto_merged.join(postalcode_venues_sorted.set_index('Postal Code'), on='PostalCode')

dwnToronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Hotel,Café,Japanese Restaurant,Diner,Electronics Store,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Clothing Store,Cosmetics Shop,Bakery,Cocktail Bar,Gastropub,Department Store,Seafood Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Bakery,Coffee Shop,Cocktail Bar,Pharmacy,Farmers Market,Cheese Shop,Pub,Beer Bar,Restaurant,Seafood Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Restaurant


Finally, let's visualize the resulting clusters


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dwnToronto_merged['Latitude'], dwnToronto_merged['Longitude'], dwnToronto_merged['Neighborhood'], dwnToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [31]:
dwnToronto_merged.loc[dwnToronto_merged['Cluster Labels'] == 0, dwnToronto_merged.columns[[0,1] + list(range(5, dwnToronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M4W,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 2


In [32]:
dwnToronto_merged.loc[dwnToronto_merged['Cluster Labels'] == 1, dwnToronto_merged.columns[[0,1] + list(range(5, dwnToronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M5S,Downtown Toronto,1,Café,Bookstore,Sandwich Place,Bar,Japanese Restaurant,Bakery,Yoga Studio,Italian Restaurant,Beer Bar,Beer Store
11,M5T,Downtown Toronto,1,Café,Bar,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Dumpling Restaurant,Burger Joint,Park,Bakery


#### Cluster 3


In [33]:
dwnToronto_merged.loc[dwnToronto_merged['Cluster Labels'] == 2, dwnToronto_merged.columns[[0,1] + list(range(5, dwnToronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Shoe Store,Restaurant
1,M5B,Downtown Toronto,2,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Hotel,Café,Japanese Restaurant,Diner,Electronics Store,Pizza Place
2,M5C,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Clothing Store,Cosmetics Shop,Bakery,Cocktail Bar,Gastropub,Department Store,Seafood Restaurant
3,M5E,Downtown Toronto,2,Bakery,Coffee Shop,Cocktail Bar,Pharmacy,Farmers Market,Cheese Shop,Pub,Beer Bar,Restaurant,Seafood Restaurant
4,M5G,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Restaurant
6,M5H,Downtown Toronto,2,Coffee Shop,Clothing Store,Café,Restaurant,Hotel,Thai Restaurant,Gym,Bar,Deli / Bodega,Cosmetics Shop
7,M5J,Downtown Toronto,2,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Restaurant,Fried Chicken Joint,Brewery,Park,Bar
8,M5K,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Bakery,Salad Place,Concert Hall
9,M5L,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Hotel,Gym,Bakery,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant
14,M4X,Downtown Toronto,2,Coffee Shop,Pizza Place,Café,Italian Restaurant,Bakery,Park,Restaurant,Pub,General Entertainment,Beer Store


#### Cluster 4


In [34]:
dwnToronto_merged.loc[dwnToronto_merged['Cluster Labels'] == 3, dwnToronto_merged.columns[[0,1] + list(range(5, dwnToronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M5V,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Boat or Ferry,Plane,Rental Car Location


#### Cluster 5


In [35]:
dwnToronto_merged.loc[dwnToronto_merged['Cluster Labels'] == 4, dwnToronto_merged.columns[[0,1] + list(range(5, dwnToronto_merged.shape[1]))]]

,PostalCode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M6G,Downtown Toronto,4,Grocery Store,Café,Park,Baby Store,Coffee Shop,Nightclub,Candy Store,Restaurant,Italian Restaurant,Concert Hall


We only have 17 postal codes in Downtown Toronto which explains why the clustered points in the visualization and the number of data points in each cluster small. Here's a few points 

- The most common venue in M4W, DownTown Toronto, which is the only postal code in Cluster 0 is Park 
- The most common venue in Cluster 1 is Café
- Cluster 2 have 12 postal codes. This is by far the largest cluster and the most common venue in this cluster is Coffee Shop
- The most common venue in cluster 4 is Airport Service and it is in M5V, Downtown Toronto
- Lastly, the cluster 5 (along with CLuster 1 and 4) only contain 1 point and the most common venue in this are is Grocery Store. 